# Examen Final
## Curso: Data Mining Tools
## Nombre: Ibrahim Imanol Jordi Arquinigo Jacinto
## Codigo: U20191e650



# Librerias

In [59]:
%pip install deap optuna kagglehub streamlit scikit-learn

In [60]:
# Manipulación de datos
import pandas as pd
import numpy as np
import os

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Preprocesamiento y modelos
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.cluster import KMeans
from sklearn.manifold import MDS
# Para trabajar con fechas y horas
from datetime import datetime
# Modelizacion y optimizacion
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from deap import base, creator, tools, algorithms
import random

import kagglehub
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
import optuna
import streamlit as st

# Cargar Datos

In [61]:
import kagglehub
import pandas as pd

# Descargar el dataset desde Kaggle
dataset_path = kagglehub.dataset_download("brendan45774/test-file")

# Ubicar el archivo CSV en la carpeta descargada
csv_file = f"{dataset_path}/tested.csv"  # Ajustar según el nombre real del archivo

# Cargar el dataset en un DataFrame de pandas
data = pd.read_csv(csv_file)

# Mostrar las primeras filas del dataset
print(data.head())


   PassengerId  Survived  Pclass  \
0          892         0       3   
1          893         1       3   
2          894         0       2   
3          895         0       3   
4          896         1       3   

                                           Name     Sex   Age  SibSp  Parch  \
0                              Kelly, Mr. James    male  34.5      0      0   
1              Wilkes, Mrs. James (Ellen Needs)  female  47.0      1      0   
2                     Myles, Mr. Thomas Francis    male  62.0      0      0   
3                              Wirz, Mr. Albert    male  27.0      0      0   
4  Hirvonen, Mrs. Alexander (Helga E Lindqvist)  female  22.0      1      1   

    Ticket     Fare Cabin Embarked  
0   330911   7.8292   NaN        Q  
1   363272   7.0000   NaN        S  
2   240276   9.6875   NaN        Q  
3   315154   8.6625   NaN        S  
4  3101298  12.2875   NaN        S  


# Descripcion de los datos

In [62]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Survived     418 non-null    int64  
 2   Pclass       418 non-null    int64  
 3   Name         418 non-null    object 
 4   Sex          418 non-null    object 
 5   Age          332 non-null    float64
 6   SibSp        418 non-null    int64  
 7   Parch        418 non-null    int64  
 8   Ticket       418 non-null    object 
 9   Fare         417 non-null    float64
 10  Cabin        91 non-null     object 
 11  Embarked     418 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 39.3+ KB


In [63]:
# Verificar la cantidad de valores nulos por columna
print("\nValores nulos por columna:")
print(data.isnull().sum())


Valores nulos por columna:
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


In [64]:
# Verificar si hay celdas con cadenas vacías ("")
print("\nCeldas con cadenas vacías:")
print((data == '').sum())


Celdas con cadenas vacías:
PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64


In [65]:
# Verificar si hay filas duplicadas
print("\nNúmero de filas duplicadas:")
print(data.duplicated().sum())


Número de filas duplicadas:
0


In [66]:
data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,0.363636,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.481622,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,0.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,0.000000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,0.000000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,1.000000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,1.000000,3.000000,76.000000,8.000000,9.000000,512.329200


In [67]:
data.describe(include=['object'])

,Name,Sex,Ticket,Cabin,Embarked
count,418,418,418,91,418
unique,418,2,363,76,3
top,"Kelly, Mr. James",male,PC 17608,B57 B59 B63 B66,S
freq,1,266,5,3,270


# Preprocesamiento

In [68]:
# Seleccionar solo las columnas relevantes
data = data[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'Survived']]

# Tratar valores nulos

# Llenar los valores nulos en 'Age' con la mediana
data['Age'] = data['Age'].fillna(data['Age'].median())

# Llenar los valores nulos en 'Fare' con la mediana
data['Fare'] = data['Fare'].fillna(data['Fare'].median())

# Llenar los valores nulos en 'Embarked' con el valor más frecuente
data['Embarked'] = data['Embarked'].fillna(data['Embarked'].mode()[0])

# Verificar nuevamente los valores nulos
print("Valores nulos después del preprocesamiento:")
print(data.isnull().sum())


# Codificar variables categóricas
label_enc = LabelEncoder()
data['Sex'] = label_enc.fit_transform(data['Sex'])
data['Embarked'] = label_enc.fit_transform(data['Embarked'])

# Separar características (X) y variable objetivo (y)
X = data.drop('Survived', axis=1)
y = data['Survived']

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar los datos para modelos sensibles a magnitudes
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


Valores nulos después del preprocesamiento:
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
Survived    0
dtype: int64


<ipython-input-68-4132c684c138>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Age'] = data['Age'].fillna(data['Age'].median())
<ipython-input-68-4132c684c138>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Fare'] = data['Fare'].fillna(data['Fare'].median())
<ipython-input-68-4132c684c138>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://panda

In [73]:
import optuna
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Función objetivo para Random Forest con regularización
def objective_rf(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 2, 20)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        random_state=42
    )

    # Validación cruzada para obtener la exactitud promedio
    cv_scores = cross_val_score(model, X, y, cv=5)
    return cv_scores.mean()

# Optimizar Random Forest
study_rf = optuna.create_study(direction='maximize')
study_rf.optimize(objective_rf, n_trials=30)

# Obtener los mejores hiperparámetros
best_params_rf = study_rf.best_params
print("Mejores hiperparámetros para Random Forest:", best_params_rf)


[I 2024-11-26 21:29:18,391] A new study created in memory with name: no-name-a03e7ecd-e327-46a5-bdb2-7c097eeed157
[I 2024-11-26 21:29:19,758] Trial 0 finished with value: 1.0 and parameters: {'n_estimators': 94, 'max_depth': 18, 'min_samples_split': 2, 'min_samples_leaf': 6}. Best is trial 0 with value: 1.0.
[I 2024-11-26 21:29:21,197] Trial 1 finished with value: 1.0 and parameters: {'n_estimators': 156, 'max_depth': 4, 'min_samples_split': 4, 'min_samples_leaf': 9}. Best is trial 0 with value: 1.0.
[I 2024-11-26 21:29:22,684] Trial 2 finished with value: 1.0 and parameters: {'n_estimators': 189, 'max_depth': 9, 'min_samples_split': 10, 'min_samples_leaf': 8}. Best is trial 0 with value: 1.0.
[I 2024-11-26 21:29:23,492] Trial 3 finished with value: 1.0 and parameters: {'n_estimators': 95, 'max_depth': 8, 'min_samples_split': 9, 'min_samples_leaf': 3}. Best is trial 0 with value: 1.0.
[I 2024-11-26 21:29:23,969] Trial 4 finished with value: 1.0 and parameters: {'n_estimators': 56, 'max

Mejores hiperparámetros para Random Forest: {'n_estimators': 94, 'max_depth': 18, 'min_samples_split': 2, 'min_samples_leaf': 6}


In [74]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

# Función objetivo para Logistic Regression con regularización
def objective_lr(trial):
    C = trial.suggest_loguniform('C', 1e-4, 1e2)  # Regularización
    solver = trial.suggest_categorical('solver', ['lbfgs', 'liblinear'])
    max_iter = trial.suggest_int('max_iter', 100, 500)

    model = LogisticRegression(C=C, solver=solver, max_iter=max_iter, random_state=42)

    # Validación cruzada para obtener la exactitud promedio
    cv_scores = cross_val_score(model, X, y, cv=5)
    return cv_scores.mean()

# Optimizar Logistic Regression
study_lr = optuna.create_study(direction='maximize')
study_lr.optimize(objective_lr, n_trials=30)

# Obtener los mejores hiperparámetros
best_params_lr = study_lr.best_params
print("Mejores hiperparámetros para Logistic Regression:", best_params_lr)


[I 2024-11-26 21:30:08,501] A new study created in memory with name: no-name-cc242247-dd2c-4e8b-9843-10d5c7f94a3a
<ipython-input-74-ff8387d87137>:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 1e-4, 1e2)  # Regularización
[I 2024-11-26 21:30:08,636] Trial 0 finished with value: 1.0 and parameters: {'C': 8.68094890414918, 'solver': 'lbfgs', 'max_iter': 168}. Best is trial 0 with value: 1.0.
<ipython-input-74-ff8387d87137>:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 1e-4, 1e2)  # Regularización
[I 2024-11-26 21:30:08,790] Trial 1 finished with value: 1.0 and parameters: {'C': 33.26298636233083, 'so

Mejores hiperparámetros para Logistic Regression: {'C': 8.68094890414918, 'solver': 'lbfgs', 'max_iter': 168}


In [88]:
# Entrenar Random Forest con los mejores hiperparámetros
optimized_rf = RandomForestClassifier(**best_params_rf, random_state=42)
optimized_rf.fit(X_train, y_train)

# Entrenar Logistic Regression con los mejores hiperparámetros
optimized_lr = LogisticRegression(class_weight='balanced', **best_params_lr, random_state=42)
optimized_lr.fit(X_train, y_train)


LogisticRegression(C=8.68094890414918, class_weight='balanced', max_iter=168,
                   random_state=42)

In [89]:
from sklearn.model_selection import cross_val_score

# Validación cruzada para Random Forest
rf_cv_scores = cross_val_score(optimized_rf, X, y, cv=5)
print(f"Exactitud promedio con validación cruzada (Random Forest): {rf_cv_scores.mean()}")

# Validación cruzada para Logistic Regression
lr_cv_scores = cross_val_score(optimized_lr, X, y, cv=5)
print(f"Exactitud promedio con validación cruzada (Logistic Regression): {lr_cv_scores.mean()}")


Exactitud promedio con validación cruzada (Random Forest): 1.0
Exactitud promedio con validación cruzada (Logistic Regression): 1.0


In [90]:
from sklearn.metrics import classification_report

# Reporte de clasificación para Random Forest
print("Reporte de clasificación (Random Forest):")
print(classification_report(y_test, y_pred_rf))

# Reporte de clasificación para Logistic Regression
print("Reporte de clasificación (Logistic Regression):")
print(classification_report(y_test, y_pred_lr))


Reporte de clasificación (Random Forest):
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        50
           1       1.00      1.00      1.00        34

    accuracy                           1.00        84
   macro avg       1.00      1.00      1.00        84
weighted avg       1.00      1.00      1.00        84

Reporte de clasificación (Logistic Regression):
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        50
           1       1.00      1.00      1.00        34

    accuracy                           1.00        84
   macro avg       1.00      1.00      1.00        84
weighted avg       1.00      1.00      1.00        84



In [91]:
# Obtener las probabilidades de los modelos
rf_probs = optimized_rf.predict_proba(X_test)[:, 1]  # Probabilidades para la clase 1 (Sobrevivió)
lr_probs = optimized_lr.predict_proba(X_test)[:, 1]  # Probabilidades para la clase 1 (Sobrevivió)

# Comparar las probabilidades
comparison = pd.DataFrame({
    'RF Prob': rf_probs,
    'LR Prob': lr_probs,
    'True Label': y_test
})
print(comparison.head())


      RF Prob   LR Prob  True Label
321  0.022527  0.001889           0
324  0.889595  0.998573           1
388  0.075511  0.001784           0
56   0.042588  0.001665           0
153  0.860238  0.997867           1


## Conclusión:
#### Ambos modelos (Random Forest y Logistic Regression) están realizando predicciones bastante alineadas, con las probabilidades de las clases coincidiendo con las etiquetas verdaderas.
#### Random Forest tiene una ligera diferencia en las probabilidades, pero sigue siendo consistente con las predicciones correctas.
#### Logistic Regression tiende a dar probabilidades muy cercanas a los valores extremos (cerca de 0 o 1), lo que indica que el modelo está bastante seguro de sus decisiones, pero en algunos casos, su sensibilidad a las predicciones puede ser demasiado alta (especialmente cuando las probabilidades son cercanas a 1).
#### Ambos modelos parecen estar funcionando bien en este pequeño conjunto de ejemplos, pero es importante asegurarse de que estos resultados se mantengan consistentes en un conjunto de datos más grande o diverso.

In [92]:
import pickle

# Guardar Random Forest optimizado
with open('optimized_rf.pkl', 'wb') as rf_file:
    pickle.dump(optimized_rf, rf_file)

# Guardar Logistic Regression optimizado
with open('optimized_lr.pkl', 'wb') as lr_file:
    pickle.dump(optimized_lr, lr_file)
